# JTC: Javascript Trust Chain JWT Header

## Use Case
When issuing x509 SPIFFE SVIDs… usually the workload receiving the SVID is at the end of a chain of certificates, only the root of which is going to necessarily be readily accessible.  You can imagine a series like:

* Spire server root cert
* K8s Cluster cert
* Node cert
* Workload cert

JWTs are extremely usable tokens. They even offer options to sign with an X509 cert.  One would like to be able to use a JWT token signed with a Workload cert to assert identity to any receiving workload that trust’s the Spire server root’s trust domain (either directly or via federation of trust domains) even if its not running in the same K8s cluster or on the same Node. 

## Existing Options

The options provided by JWS RFC 7515 offers a number of choices for header fields to convey information about the signing X509 certificate:

* jwk - JWK encoded certificate or certificate chain
* jku - a url from which to fetch JWK encoded certificate or certificate chain
* x5c - JSON array containing the x509 encoded certificate chain
* x5u - url from which to fetch x509 encoded certificate or certificate chain

## Problems

### jku/x5u
jku/x5u urls cannot be easily used to send a receiver of a JWT token to retrieve the Workload cert, which may be inaccessible by the recipient by virtue of being in a different network domain.
Additionally, url based retrieval can greatly increase latency of validating JWT tokens.

### jwk/x5c
jwk/x5c by presenting the full chain of the certificates used to sign the JWT tempts the recipient to validate the token without validating the certificate chain.  This is error prone.  TLS has similar issues, but the stock TLS libraries have all ‘failed safe’ for this problem.  JWT is much looser and thus more dangerous.

## Proposed Solution
Introduce a new JWT header field:

jtc - JSON trust chain.  The ‘jtc’ header is a JSON array of JWTs of the form:

```
{  // JWT header
    alg: // algorithm used for signature
    typ:
    enc: // algorithm used for encryption
    // pick one of x5t,x5u,jku but *not* x5c or jwk
    // x5t - fingerprint or thumbnail of x509 key
    // x5u - url of x509 key
    // jku - url of jwk key
}
{  // JWT payload
    x5e: // x509 certificate encrypted by key indicated in header
}
{ // JWT signature
}
```

For our example of:
* Spire server root cert
* K8s cluster cert
* Node cert
* Workload cert

Spire server root could provide the K8s cluster with a ‘jtc’ using x5t for its finger print.

K8s cluster cert could provide the Node with a ‘jtc’ containing the Spire root’s ‘jtc’ with its own ‘jtc’ JWT appended.

The Node could issue a JWT SVID to the Workload with a ‘jtc’ containing the K8s cluster’s ‘jtc’ with its own ‘jtc’ JWT appended.

The workload can then add the ‘jtc’ to JWT tokens it’s signing with its workload X509 cert.  

Any other workload that receives that JWT token that is either in the trust domain of the Spire server root, or federated with the Spire server root can then validly reconstruct the certificate needed to validate the JWT’s signature because:
The receiving workload should have the Spire root’s public key for its fingerprint as a known trusted authority.  It can use that public key to decrypt the K8s cluster’s X509 public key from the Spire root’s ‘jtc’ JWT.

Now the workload has the K8s cluster’s public key, and knows it can trust it.

This can decrypt the Node’s X509 cert using the K8s cluster’s public key

And then the Workloads X509 cert using the Node’s public key

This solves the issue with jwk/x5c certificate chains because it is *not possible* for the receiving workload to verify the JWT *unless* it already has (and thus presumably trusts) a public key matching the x5t fingerprint of one of the JWTs in the ‘jtc’ chain or is able to retrieve one of them from the jku/x5u.

Caching and performance improvements
Once the receiving workload has processed the ‘jtc’ from the Workload, it now has and trusts the X509 public keys for the K8s cluster and the Node, and so  can begin unpacking any ‘jtc’s from that K8s Cluster or Node without having to re-decrypt their public keys from the chain.



  


 




In [1]:
{  // JWT header
    alg: // algorithm used for signature
    typ:
    enc: // algorithm used for encryption
    // pick one of x5t,x5u,jku but *not* x5c or jwk
    // x5t - fingerprint or thumbnail of x509 key
    // x5u - url of x509 key
    // jku - url of jwk key
}
{  // JWT payload
    x5e: // x509 certificate encrypted by key indicated in header
}
{ // JWT signature
}


In [2]:
import(
    "crypto/rand"
    "crypto/rsa"
    "crypto/sha256"
    "crypto/x509"
    "fmt"
    "os"
    
    "github.com/fkautz/pubdecrypt"
)

In [3]:
secretMessage := "secret message"

In [4]:
// The GenerateKey method takes in a reader that returns random bits, and
// the number of bits
privateKey, err := rsa.GenerateKey(rand.Reader, 2048)
if err != nil {
    panic(err)
}

// The public key is a part of the *rsa.PrivateKey struct
publicKey := privateKey.PublicKey

In [5]:
encryptedMessage, err := pubdecrypt.EncryptPKCS1v15(rand.Reader, privateKey, []byte(secretMessage))

In [6]:
fmt.Println(encryptedMessage)
fmt.Println(err)

[59 31 27 139 17 23 45 84 138 32 233 193 107 42 46 198 203 211 193 200 205 148 115 241 77 26 52 201 218 218 238 11 65 0 108 196 201 229 80 173 243 72 74 224 86 112 72 89 33 225 212 180 43 223 11 95 187 47 146 165 90 141 226 108 56 211 86 92 57 85 29 13 104 40 99 84 171 47 123 233 45 236 91 136 58 75 41 90 195 65 34 95 38 77 135 22 28 157 150 186 57 6 144 220 114 138 240 135 116 214 67 179 204 244 33 107 108 188 229 231 53 159 211 43 121 84 9 111 143 215 9 190 143 242 31 140 170 201 239 16 166 101 222 133 46 115 38 27 177 248 28 120 126 184 20 99 91 120 203 143 156 57 178 216 88 102 255 250 231 25 228 95 26 81 109 170 252 192 99 147 241 212 84 92 6 16 134 233 143 100 29 175 178 36 59 53 37 126 121 244 123 211 207 42 41 80 142 110 188 199 233 249 105 97 36 135 158 129 133 188 13 218 103 146 225 52 61 211 39 155 46 42 237 135 111 244 142 76 206 125 36 167 140 138 135 198 100 186 129 222 180 119 93 69 212 102]
<nil>


6 <nil>

In [7]:
// Unsign verifies the message using a rsa-sha256 signature
// func (r *rsaPublicKey) Unsign(message []byte, sig []byte) error {
// 	h := sha256.New()
// 	h.Write(message)
// 	d := h.Sum(nil)
// 	return rsa.VerifyPKCS1v15(r.PublicKey, crypto.SHA256, d, sig)
// }
// rsa.VerifyPKCS1v15(&privateKey.PublicKey, crypto.Hash(0), encryptedBytes, []byte(secretMessage))
decryptedMessage, err := pubdecrypt.DecryptPKCS1v15(rand.Reader, &publicKey, encryptedMessage)

In [8]:
fmt.Println(string(decryptedMessage))
fmt.Println(err)

secret message
<nil>


6 <nil>